Entrene un autoencoder para obtener una representación de baja dimensionalidad de las imágenes de MNIST. Use dichas representaciones para entrenar un perceptrón multicapa como clasificador. ¿Cuál es el tiempo de entrenamiento y la exactitud del clasificador obtenido cuando parte de la representación del autoencoder, en comparación con lo obtenido usando las imágenes originales?

En este caso se nos permitió usar Pytorch para la parte de código. La idea es definir la estructura general de la red y luego ir modificandola hasta lograr la mejor performance. 

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
import matplotlib.pyplot as plt

# Elegir dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Transformación
transform = transforms.ToTensor()

# Dataset MNIST
train_dataset = datasets.MNIST(root="data", train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root="data", train=False, download=True, transform=transform)

# minibatch
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=len(test_dataset))

Device: cpu


Primero una función que entrena y grafica

In [12]:
def train_and_evaluate(model, train_loader, test_loader,
                       criterion, optimizer, device,
                       num_epochs=50, patience=5, min_delta=1e-4):
    
    model = model.to(device)
    best_loss = float("inf")
    epochs_no_improve = 0
    
    history = {"train_loss": [], "test_loss": [], "accuracy": []}

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(model)
    print(f"Parámetros entrenables: {total_params:,}")

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for x,y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        history["train_loss"].append(train_loss)

        # Evaluación
        model.eval()
        test_loss = 0.0
        correct, total = 0, 0

        with torch.no_grad():
            for x,y in test_loader:
                x, y = x.to(device), y.to(device)
                outputs = model(x)
                test_loss += criterion(outputs, y).item()
                _, pred = outputs.max(1)
                total += y.size(0)
                correct += pred.eq(y).sum().item()

        test_loss /= len(test_loader)
        accuracy = 100 * correct / total

        history["test_loss"].append(test_loss)
        history["accuracy"].append(accuracy)

        print(f"Epoch {epoch+1} | Train Loss {train_loss:.4f} | "
              f"Test Loss {test_loss:.4f} | Acc {accuracy:.2f}%")

        # Early stopping
        if test_loss < best_loss - min_delta:
            best_loss = test_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print("✔ Early stopping activado")
            break

    # Gráficos
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history["train_loss"], label="Train")
    plt.plot(history["test_loss"], label="Test")
    plt.title("Loss")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(history["accuracy"], label="Accuracy")
    plt.title("Accuracy (%)")
    plt.legend()
    plt.show()

    return history


In [13]:
def train_autoencoder(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=10):
    train_losses = []
    test_losses = []
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for x, _ in train_loader:
            x = x.to(device)
            x_flat = x.view(x.size(0), -1)     # 🔹 aplanar
            optimizer.zero_grad()
            outputs = model(x_flat)
            loss = criterion(outputs, x_flat)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_losses.append(running_loss / len(train_loader))

        # evaluación
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for x, _ in test_loader:
                x = x.to(device)
                x_flat = x.view(x.size(0), -1)
                outputs = model(x_flat)
                loss = criterion(outputs, x_flat)
                test_loss += loss.item()
        test_losses.append(test_loss / len(test_loader))

        print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}")

    return train_losses, test_losses


La primer estructura que quiero hacer es el autoencoder. Vo a elegir una arquitectura simétrica con una reducción del 80% de tamaño de dimensión de entrada, de acuerdo al principio de Pareto, que dice que el 80% de las cosas son causadas por el 20%. Esto lo extrapolo a energía y varianza y listo. 

Entramos con $28*28$, luego a $28*28*0.5 $y luego a $0.2*28*28$. De ahí sube de vuelta

In [14]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 392)
        self.fc2 = nn.Linear(392, 156)
        self.fc3 = nn.Linear(156, 392)
        self.fc4 = nn.Linear(392, 28*28)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

model_cnn = Autoencoder().to(device)


In [ ]:
# -------- Entrenamiento --------
criterion = nn.BCELoss()
optimizer = optim.Adam(model_cnn.parameters(), lr=1e-3)

train_losses, test_losses = train_autoencoder(
    model_cnn, train_loader, test_loader,
    criterion, optimizer, device,
    num_epochs=30
)


# -------- Gráficos --------
plt.figure(figsize=(8,4))
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Pérdida del Autoencoder')
plt.show()


# -------- Visualizar reconstrucciones --------
model_cnn.eval()
with torch.no_grad():
    for x, _ in test_loader:
        x = x.to(device)
        x_flat = x.view(x.size(0), -1)
        outputs = model_cnn(x_flat)
        break  # solo un batch

# convertir a imágenes
x = x.cpu().view(-1, 1, 28, 28)
outputs = outputs.cpu().view(-1, 1, 28, 28)

# mostrar algunas imágenes
n = 8
plt.figure(figsize=(15,4))
for i in range(n):
    # original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x[i][0], cmap="gray")
    plt.title("Original")
    plt.axis("off")

    # reconstruida
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(outputs[i][0], cmap="gray")
    plt.title("Reconstruida")
    plt.axis("off")

plt.show()

Epoch [1/30] - Train Loss: 0.0693, Test Loss: 0.0691


1570 parámetros no congelados, la capa de 156 a 10 + bias

In [ ]:
class ClassifierFromEncoder(nn.Module):
    def __init__(self, autoencoder, num_classes=10):
        super().__init__()
        # acá tomamos las capas del otro modelo y las congelamos
        self.encoder_fc1 = autoencoder.fc1
        self.encoder_fc2 = autoencoder.fc2
        for param in self.encoder_fc1.parameters():
            param.requires_grad = False
        for param in self.encoder_fc2.parameters():
            param.requires_grad = False
        self.classifier = nn.Linear(156, num_classes) # lo más simple posible

    def forward(self, x):
        x = x.view(x.size(0), -1)
        with torch.no_grad():
            x = F.relu(self.encoder_fc1(x))
            encoded = F.relu(self.encoder_fc2(x))
        logits = self.classifier(encoded)
        return logits

classifier_model = ClassifierFromEncoder(model_cnn).to(device)
criterion_cls = nn.CrossEntropyLoss()
optimizer_cls = optim.Adam(classifier_model.classifier.parameters(), lr=1e-3)

# ------------------------
# Entrenamiento Clasificador
# ------------------------
num_epochs_cls = 30
for epoch in range(num_epochs_cls):
    classifier_model.train()
    running_loss = 0
    correct, total = 0, 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer_cls.zero_grad()
        outputs = classifier_model(x)
        loss = criterion_cls(outputs, y)
        loss.backward()
        optimizer_cls.step()
        running_loss += loss.item()
        _, pred = outputs.max(1)
        correct += pred.eq(y).sum().item()
        total += y.size(0)
    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs_cls} | Loss={running_loss/len(train_loader):.4f} | Acc={train_acc:.2f}%")

# ------------------------
# Evaluación en Test
# ------------------------
classifier_model.eval()
correct, total = 0, 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        outputs = classifier_model(x)
        _, pred = outputs.max(1)
        correct += pred.eq(y).sum().item()
        total += y.size(0)
print(f"Test Accuracy: {100 * correct/total:.2f}%")

Epoch 1/10 | Loss=0.6868 | Acc=80.41%
Epoch 2/10 | Loss=0.3539 | Acc=89.82%
Epoch 3/10 | Loss=0.3204 | Acc=90.53%
Epoch 4/10 | Loss=0.3070 | Acc=90.93%
Epoch 5/10 | Loss=0.2998 | Acc=91.19%
Epoch 6/10 | Loss=0.2950 | Acc=91.32%
Epoch 7/10 | Loss=0.2916 | Acc=91.31%
Epoch 8/10 | Loss=0.2891 | Acc=91.43%
Epoch 9/10 | Loss=0.2872 | Acc=91.48%
Epoch 10/10 | Loss=0.2845 | Acc=91.59%
Test Accuracy: 92.13%
